### Workflow
1. Extract the start and end vertices from each path feature in the LCP pipeline network
1. Split the LCP lines at the vertices --> "lcp_edges"
1. Extract the from- and to- vertices associated with each segment
1. Spatially join the "lcp_edge" ID to the from- and the to- vertices feature classes
1. Add a unique value to each node
1. Split the path features by these nodes; these will become our edges
1. Spatially join the node feature IDs to the 

In [1]:
#Imports
import arcpy
import numpy as np
import pandas as pd

In [2]:
#Set arcpy environments
arcpy.env.workspace = '..\\scratch\\scratch.gdb'
arcpy.env.scratchWorkspace = '..\\scratch\\scratch.gdb'
arcpy.env.overwriteOutput = True

In [3]:
#Set inputs
lcp_fc = '..\\scratch\\LCPNetwork.shp'                          # LCP feature class
bgprj_fc = '..\\OptimizeViewer\\OptimizeViewer.gdb\\Biogas_prj' # Biogas source locations, w/attributes

In [4]:
#Convert LCP vertices to points
lcp_nodes = arcpy.management.FeatureVerticesToPoints(in_features=lcp_fc,
                                                    out_feature_class='memory/LCP_vertices',
                                                    point_location='BOTH_ENDS').getOutput(0)

In [22]:
#Split the LCP at the vertices
if arcpy.Exists('LCP_edges'): arcpy.management.Delete('LCP_edges')
lcp_split = arcpy.management.SplitLineAtPoint(in_features=lcp_fc,
                                              point_features=lcp_nodes,
                                              out_feature_class='LCP_edges_temp',
                                              search_radius='10 meters').getOutput(0)

#Remove features with no length
lcp_edges = arcpy.analysis.Select(in_features=lcp_split,
                                  out_feature_class='LCP_edges',
                                  where_clause="Shape_Length > 0"
                                 ).getOutput(0)

#Remove the temp dataset
arcpy.management.Delete('LCP_edges_temp')

#Add an edge ID field
arcpy.management.AddField(in_table=lcp_edges,field_name='Edge_ID',field_type='SHORT')
arcpy.management.CalculateField(in_table=lcp_edges,field='Edge_ID',expression='!OBJECTID!')

<Result '..\\scratch\\scratch.gdb\\LCP_edges'>

In [26]:
#Re-convert the vertices to points
lcp_frNodes = arcpy.management.FeatureVerticesToPoints(in_features=lcp_edges,
                                                       out_feature_class='LCP_frNodes',
                                                       point_location='END').getOutput(0)

lcp_toNodes = arcpy.management.FeatureVerticesToPoints(in_features=lcp_edges,
                                                       out_feature_class='LCP_toNodes',
                                                       point_location='START').getOutput(0)

In [30]:
#Spatially join the edge IDs to each to and from node
if arcpy.Exists('LCP_nodes'): arcpy.management.Delete('LCP_nodes')
lcp_Nodes = arcpy.analysis.SpatialJoin(target_features="LCP_frNodes", 
                                       join_features="LCP_toNodes", 
                                       out_feature_class="LCP_nodes", 
                                       join_operation="JOIN_ONE_TO_MANY", 
                                       join_type="KEEP_ALL", 
                                       field_mapping=('Ids "Ids" true true false 8 Double 0 0,First,#,LCP_frNodes,Ids,-1,-1;'+
                                                      'FromID "FromID" true true false 2 Short 0 0,First,#,LCP_frNodes,Edge_ID,-1,-1;'+
                                                      'ToID "ToID" true true false 2 Short 0 0,First,#,LCP_toNodes,Edge_ID,-1,-1'), 
                                       match_option="INTERSECT").getOutput(0)

The `lcp_edges` feature class now includes two features for each segment in the LCP network. The two segments share the same `TARGET_FID` value, and the `PtID` value corresponds to the node on either side. So we can construct an edge list by pivoting the attribute table: row  = `TARGET_FID` and column = `PtID`...

In [34]:
#Convert the table to a numpy array
arr_nodes = arcpy.da.TableToNumPyArray(in_table=lcp_Nodes,field_names=['FromID','ToID'],
                                      null_value={'FromID':-1,'ToID':-1})

#And to a Pandas dataframe
df_edges = pd.DataFrame(arr_nodes)
df_edges.head()

,FromID,ToID
0,1,3
1,2,4
2,3,2
3,4,-1
4,5,13


In [41]:
#Spatially join the Node IDs to the BG points
if arcpy.Exists('BG_sites'): arcpy.management.Delete('BG_sites')
bg_sites = arcpy.analysis.SpatialJoin(target_features=bgprj_fc,
                           join_features=lcp_edges,
                           out_feature_class='BG_sites',
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_COMMON", 
                           match_option="WITHIN_A_DISTANCE", 
                           search_radius="500 Meters").getOutput(0)

In [43]:
#Convert to Pnadas dataframe (via Numpy array)
dfSites = pd.DataFrame(arcpy.da.TableToNumPyArray(in_table=bg_sites,
                                                  field_names=['Edge_ID','Total_Potential_Methane_Yield_1']))
dfSites.head()

,Edge_ID,Total_Potential_Methane_Yield_1
0,105,1895.402385
1,555,203.029667
2,105,80.431709
3,211,991.060070
4,119,1684.802120


In [44]:
#Join methane yield to edge list
df_edgelist2 = pd.merge(left=df_edges,left_on='FromID',right=dfSites,right_on='Edge_ID',how='left')
df_edgelist2.fillna(0,inplace=True)
df_edgelist2.drop('Edge_ID',axis=1,inplace=True)
df_edgelist2.columns = ['From_node','To_node','Yield_scf_h']
df_edgelist2.head()

,From_node,To_node,Yield_scf_h
0,1,3,5450.830387
1,2,4,1874.747050
2,3,2,0.000000
3,4,-1,0.000000
4,5,13,5237.045262


In [45]:
#Write the edge list to a csv
#arcpy.management.CopyFeatures(in_features=lcp_nodes,out_feature_class='LCP_nodes')
df_edgelist2.to_csv('../scratch/EdgeList.csv',index=False)